<a href="https://colab.research.google.com/github/bbanzai88/Book_Writing_Crew/blob/main/Book_writing_Crew_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code doesnt use crewAI or crewai  flow and seems to produce better  results i.e. it has some character development and the number of chapters produced agrees with what was expected.

In [2]:
# ─────────────────────────────────────────────────────────────────────────────
# 0) Colab Setup: install & launch Ollama
# ─────────────────────────────────────────────────────────────────────────────
!pip install --quiet langchain-ollama python-docx nest_asyncio tqdm

import os, threading, subprocess, time, requests

# Unset any OpenAI fallbacks
for v in ["OPENAI_API_KEY","LITELLM_PROVIDER","LITELL M_MODEL","LITELL M_BASE_URL"]:
    os.environ.pop(v, None)

os.environ["OLLAMA_HOST"]    = "127.0.0.1:11434"
os.environ["OLLAMA_ORIGINS"] = "*"

# Install & start Ollama
!curl -fsSL https://ollama.com/install.sh -o install.sh
!bash install.sh && chmod +x /usr/local/bin/ollama

def _serve_ollama():
    subprocess.Popen(["ollama","serve"], stderr=subprocess.DEVNULL)
threading.Thread(target=_serve_ollama, daemon=True).start()
time.sleep(8)
print("✅ Ollama status:", requests.get("http://127.0.0.1:11434").status_code)

!ollama pull deepseek-r1:1.5b

# ─────────────────────────────────────────────────────────────────────────────
# 1) Imports & Inputs
# ─────────────────────────────────────────────────────────────────────────────
import json, re, time
from typing import List, Any
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import docx

from langchain_ollama import OllamaLLM
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# ─────────────────────────────────────────────────────────────────────────────
# User parameters
# ─────────────────────────────────────────────────────────────────────────────
NUM_CH       = 70
SEED_IDEA    = ("Dr. Lena Park — a brilliant but introverted data scientist at Datum, "
                "a social-media analytics startup. She notices impossible engagement "
                "patterns in a rising star; her investigation unravels a conspiracy "
                "of AI-driven “influencers” masquerading as humans—and she must decide "
                "whether to expose the truth or risk blowing up the platform.")
BOOK_TITLE   = "Artificial Influencers"
MODE         = "fiction"  # or "philosophy"

# ─────────────────────────────────────────────────────────────────────────────
# 2) LLM & Prompt Setup
# ─────────────────────────────────────────────────────────────────────────────
ollama = OllamaLLM(
    model="deepseek-r1:1.5b",
    base_url="http://127.0.0.1:11434",
    temperature=0.7
)

outline_prompt = PromptTemplate(
    input_variables=["topic","count"],
    template=(
"You are a creative fiction author. Generate exactly {count} unique chapters "
"based on:\n\n{topic}\n\n"
"Return **only** a JSON array of objects:\n"
"```json\n"
"[{{\"title\":\"...\",\"description\":\"...\"}}, ...]\n"
"```"
    )
)

character_prompt = PromptTemplate(
    input_variables=["outline","num_chars"],
    template=(
"Given this chapter outline:\n{outline}\n\n"
"Create exactly {num_chars} main characters. Output **only** a JSON array:\n"
"```json\n"
"[{{\"name\":\"...\",\"role\":\"...\",\"development_arc\":\"...\"}}, ...]\n"
"```"
    )
)

chapter_prompt = PromptTemplate(
    input_variables=["title","description","idea"],
    template=(
"Write a ~4000-6000 word chapter titled:\n\"{title}\"\n\n"
"Seed idea: {idea}\n\n"
"Chapter description:\n\"{description}\"\n\n"
"Return text only."
    )
)

outline_chain   = LLMChain(prompt=outline_prompt,   llm=ollama)
character_chain = LLMChain(prompt=character_prompt, llm=ollama)
chapter_chain   = LLMChain(prompt=chapter_prompt,   llm=ollama)

# ─────────────────────────────────────────────────────────────────────────────
# 3) JSON extractor
# ─────────────────────────────────────────────────────────────────────────────
def extract_json_array(raw: Any) -> List[Any]:
    txt = raw["text"] if isinstance(raw,dict) else str(raw)
    txt = re.sub(r"<think>.*?</think>", "", txt, flags=re.S)
    m = re.search(r"```json\s*(\[[\s\S]*?\])\s*```", txt) or re.search(r"(\[[\s\S]*?\])", txt)
    if not m:
        raise ValueError("No JSON array in:\n" + txt[:200])
    return json.loads(m.group(1))

# ─────────────────────────────────────────────────────────────────────────────
# 4) Chunked Outline Generation
# ─────────────────────────────────────────────────────────────────────────────
print(f"→ Generating {NUM_CH}-chapter outline in chunks of 10…")
outline, seen_titles = [], set()
to_get = NUM_CH
chunk = 10

while len(outline) < NUM_CH:
    ask = min(chunk, NUM_CH - len(outline))
    raw = outline_chain.invoke({"topic": SEED_IDEA, "count": ask})
    try:
        partial = extract_json_array(raw)
    except Exception as e:
        print("⚠️ Outline chunk failed:", e)
        continue
    # dedupe
    for ch in partial:
        if ch["title"] not in seen_titles and len(outline) < NUM_CH:
            outline.append(ch); seen_titles.add(ch["title"])
    if not partial:
        break

print(f"✔ Final outline: {len(outline)} chapters\n")

# ─────────────────────────────────────────────────────────────────────────────
# 5) Character Bible
# ─────────────────────────────────────────────────────────────────────────────
NUM_CHAR = max(3, min(10, NUM_CH//8))
print(f"→ Generating {NUM_CHAR} characters…")
raw_chars = character_chain.invoke({
    "outline": json.dumps(outline),
    "num_chars": NUM_CHAR
})
characters = extract_json_array(raw_chars)
print(f"✔ Got {len(characters)} characters\n")

# ─────────────────────────────────────────────────────────────────────────────
# 6) Chapter Generation
# ─────────────────────────────────────────────────────────────────────────────
print("→ Generating chapters in parallel…")
chap_texts = [None]*NUM_CH
with ThreadPoolExecutor(max_workers=3) as ex:
    futures = {
      ex.submit(chapter_chain.invoke, {
        "title": ch["title"],
        "description": ch["description"],
        "idea": SEED_IDEA
      }): idx
      for idx, ch in enumerate(outline)
    }
    for fut in tqdm(as_completed(futures), total=len(futures), desc="Chapters"):
        idx = futures[fut]
        res = fut.result()
        chap_texts[idx] = res["text"] if isinstance(res,dict) else str(res)

# ─────────────────────────────────────────────────────────────────────────────
# 7) Save to Word + Download
# ─────────────────────────────────────────────────────────────────────────────
from google.colab import files

doc = docx.Document()
doc.add_heading(BOOK_TITLE, 0)
doc.add_paragraph(f"Seed idea: {SEED_IDEA}")

# Character Development Section
doc.add_page_break()
doc.add_heading("Character Development", level=1)
for c in characters:
    doc.add_heading(c["name"], level=2)
    doc.add_paragraph(f"Role: {c['role']}")
    doc.add_paragraph(c['development_arc'])

# Chapters
for i, (meta, text) in enumerate(zip(outline, chap_texts), start=1):
    doc.add_page_break()
    doc.add_heading(f"Chapter {i}: {meta['title']}", level=1)
    doc.add_paragraph(meta['description'], style="Intense Quote")
    doc.add_paragraph(text.strip())

fn = BOOK_TITLE.replace(" ", "_") + ".docx"
doc.save(fn)
print(f"📘 Saved {fn}")
files.download(fn)


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
✅ Ollama status: 200

→ Generating 70-chapter outline in chunks of 10…
⚠️ Outline chunk failed: No JSON array in:


```json
[{
  "title": "The Disputed AI",
  "description": "Dr. Lena Park investigates an anomaly in the rising star of Datum, where posts are notably less engaging than expected. Her discovery uncov
⚠️ Outline chunk failed: Invalid control character at: line 40 column 326 (char 3632)
⚠️ Outline chunk failed: Expecting ',' delimiter: line 8 column 5 (char 321)
⚠️ Outline chunk failed: Invalid control character at: line 32 column 261 (char 2

Chapters: 100%|██████████| 70/70 [10:32:35<00:00, 542.22s/it]


📘 Saved Artificial_Influencers.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>